# Python - Garbage Collection

---

Python's garbage collection (GC) complements reference counting with a generational, mark-and-sweep approach to handle circular references. Reference counting deallocates objects immediately when refcount hits zero, but fails on cycles (A->B->A). GC detects and collects these using three generations (0,1,2) based on object age.

**Reference Counting (Primary)**
* Every object tracks reference count via `ob_refcnt`
* Count +1 on new reference, -1 on deletion, when count == 0: immediate deallocation
* Fails on circular references (A -> B -> A)


**Generational GC (Cycle Detection)**
* 3 Generations: Gen0 (new), Gen1, Gen2
* Survivors promoted: Gen0 -> 1 -> 2

**Mark-and-Sweep Algorithm Steps**
* Trigger: `(allocs - deallocs) > threshold per generation`
* Mark Phase:
    * Start from roots (globals, stack, registers)
    * Traverse reachable container objects -> mark as "reachable"
    * Unmarked = tentatively unreachable (cycles)
* Sweep Phase:
    * Process weakrefs + finalizers on unreachable objects
    * Deallocate confirmed unreachable objects
    * Survivors promoted to next generation

In [1]:
import gc
import logging
import sys
import weakref

In [2]:
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format="%(message)s")

## Reference Counting

In [3]:
lst = [1, 2, 3]
log.info(f"Refcount lst: {sys.getrefcount(lst)}")

lst_alias = lst
log.info(f"Refcount after alias: {sys.getrefcount(lst)}")

del lst
log.info(f"Refcount after del lst: {sys.getrefcount(lst_alias)}")

Refcount lst: 2


Refcount after alias: 3


Refcount after del lst: 2


## Circular Reference

In [4]:
class Node:
    def __init__(self, value):
        self.value = value
        self.next = None

    def __del__(self):
        log.info(f"Node {self.value} COLLECTED")


# Create a strong reference cycle
n1 = Node(10)
n2 = Node(20)
n1.next = n2  # Strong ref
n2.next = n1  # Strong ref - creates cycle!

weak_n1 = weakref.ref(n1)  # Weak ref to check liveness later
del n1, n2
gc.collect()
log.info(f"Strong cycle: weak_n1 alive? {weak_n1() is not None}")

Node 10 COLLECTED


Node 20 COLLECTED


Strong cycle: weak_n1 alive? False
